In [ ]:
from google.colab import drive
drive.mount('/content/drive')


FOLDERNAME = "Computer Vision Project/kits23"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1tnt3cKRdWn9lFKrM82KlrZH6CO5EgchD/Computer Vision Project/kits23


In [ ]:
!pip install -q "monai-weekly[nibabel, tqdm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 96.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from monai.inferers import sliding_window_inference
from monai.networks.nets import SegResNet

# **Utils**

In [ ]:
class UNet3D(nn.Module):
    def __init__(self):
        super(UNet3D, self).__init__()
        # Encoder
        self.enc1 = self.conv_block(1, 32)
        self.enc2 = self.conv_block(32, 64)
        self.enc3 = self.conv_block(64, 128)
        self.enc4 = self.conv_block(128, 256)
        self.enc5 = self.conv_block(256, 512)
        self.pool = nn.MaxPool3d(2)

        # Decoder
        self.up5 = nn.ConvTranspose3d(512, 256, kernel_size=2, stride=2)
        self.dec5 = self.conv_block(512, 256)
        self.up4 = nn.ConvTranspose3d(256, 128, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(256, 128)
        self.up3 = nn.ConvTranspose3d(128, 64, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(128, 64)
        self.up2 = nn.ConvTranspose3d(64, 32, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(64, 32)
        self.dec1 = nn.Conv3d(32, 4, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))

        # Decoder with skip connections
        dec5 = self.up5(enc5)
        dec5 = torch.cat([dec5, enc4], dim=1)
        dec5 = self.dec5(dec5)

        dec4 = self.up4(dec5)
        dec4 = torch.cat([dec4, enc3], dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.up3(dec4)
        dec3 = torch.cat([dec3, enc2], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.up2(dec3)
        dec2 = torch.cat([dec2, enc1], dim=1)
        dec2 = self.dec2(dec2)

        # Output layer
        dec1 = self.dec1(dec2)
        return dec1

In [ ]:
class CTTestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image']
        label_path = self.data.iloc[idx]['label']

        image = torch.load(image_path)
        label = torch.load(label_path)

        if self.transform:
            image, label = self.transform(image, label)

        return image, label

In [ ]:
def test_model(model, test_dataloader, device):
    model.to(device)
    model.eval()
    test_dice = []
    test_dice_kidney = []
    test_dice_tumor = []
    test_dice_cyst = []

    with torch.no_grad():
        for test_images, test_labels in tqdm(test_dataloader):
            test_images, test_labels = test_images.to(device), test_labels.to(device)
            test_outputs = sliding_window_inference(
                inputs=test_images,
                roi_size=(128, 128, 128),
                sw_batch_size=1,
                predictor=model,
            )

            test_outputs = F.softmax(test_outputs, dim=1)
            test_outputs = torch.argmax(test_outputs, dim=1)

            dice = (2. * ((test_outputs != 0) & (test_labels != 0)).sum()) / ((test_outputs != 0).sum() + (test_labels != 0).sum() + 1e-6)
            dice_kidney = (2. * ((test_outputs == 1) & (test_labels == 1)).sum()) / ((test_outputs == 1).sum() + (test_labels == 1).sum() + 1e-6)
            dice_tumor = (2. * ((test_outputs == 2) & (test_labels == 2)).sum()) / ((test_outputs == 2).sum() + (test_labels == 2).sum() + 1e-6)

            test_dice.append(dice.item())
            test_dice_kidney.append(dice_kidney.item())
            test_dice_tumor.append(dice_tumor.item())

            print(f'Test Dice: {dice.item():.4f}')
            print(f'Test Dice Kidney: {dice_kidney.item():.4f}')
            print(f'Test Dice Tumor: {dice_tumor.item():.4f}')

            if torch.max(torch.unique(test_labels)).item() == 3:
              dice_cyst = (2. * ((test_outputs == 3) & (test_labels == 3)).sum()) / ((test_outputs == 3).sum() + (test_labels == 3).sum() + 1e-6)
              test_dice_cyst.append(dice_cyst.item())
              print(f'Test Dice Crystal: {dice_cyst.item():.4f}')

    test_avg_dice = sum(test_dice) / len(test_dice)
    test_avg_dice_kidney = sum(test_dice_kidney) / len(test_dice_kidney)
    test_avg_dice_tumor = sum(test_dice_tumor) / len(test_dice_tumor)
    test_avg_dice_cyst = sum(test_dice_cyst) / len(test_dice_cyst)

    print(f'Dice: {test_avg_dice:.4f}')
    print(f'Dice Kidney: {test_avg_dice_kidney:.4f}')
    print(f'Dice Tumor: {test_avg_dice_tumor:.4f}')
    print(f'Dice Cyst: {test_avg_dice_cyst:.4f}')

    return test_avg_dice, test_avg_dice_kidney, test_avg_dice_tumor, test_avg_dice_cyst

# **Evaluate**

In [ ]:
test_df = pd.read_csv('../preprocessed_test.csv')
test_dataset = CTTestDataset(test_df)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
model = SegResNet(
    blocks_down=[1, 2, 2, 4, 4],
    blocks_up=[1, 1, 1, 1],
    init_filters=16,
    in_channels=1,
    out_channels=4,
    dropout_prob=0.2,
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint_path = '/content/drive/MyDrive/Computer Vision Project/SegResNet3D_128_2nd/checkpoint_SegResNet_epoch_5.pth'
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

snn_dice_score, snn_dice_score_kidney, snn_dice_score_tumor, snn_dice_score_cyst = test_model(model, test_dataloader, device)

<ipython-input-16-c0a8ceff184e>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)
  0%|          | 0/98 [00:00<

Test Dice: 0.6055
Test Dice Kidney: 0.6035
Test Dice Tumor: 0.3363
Test Dice Crystal: 0.0000


  2%|▏         | 2/98 [00:48<42:19, 26.45s/it]

Test Dice: 0.7315
Test Dice Kidney: 0.7905
Test Dice Tumor: 0.2560
Test Dice Crystal: 0.0000


  3%|▎         | 3/98 [00:58<30:20, 19.16s/it]

...

Test Dice: 0.7064
Test Dice Kidney: 0.7728
Test Dice Tumor: 0.4815


 99%|█████████▉| 97/98 [34:57<00:23, 23.74s/it]

Test Dice: 0.9228
Test Dice Kidney: 0.8176
Test Dice Tumor: 0.0187
Test Dice Crystal: 0.0000


100%|██████████| 98/98 [35:22<00:00, 21.66s/it]

Test Dice: 0.8123
Test Dice Kidney: 0.7527
Test Dice Tumor: 0.0000
Dice: 0.7973
Dice Kidney: 0.8027
Dice Tumor: 0.1316
Dice Cyst: 0.0000


In [ ]:
model = UNet3D()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

checkpoint_path = '/content/drive/MyDrive/Computer Vision Project/Unet3D_128_2nd/checkpoint_UNet_epoch_5.pth'
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

dice_score, dice_score_kidney, dice_score_tumor, dice_score_cyst = test_model(model, test_dataloader, device)

<ipython-input-10-c7b0913cafee>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)
  0%|          | 0/98 [00:00<?

Test Dice: 0.5485
Test Dice Kidney: 0.5253
Test Dice Tumor: 0.2562
Test Dice Crystal: 0.0000


  2%|▏         | 2/98 [00:15<13:30,  8.44s/it]

Test Dice: 0.8230
Test Dice Kidney: 0.8179
Test Dice Tumor: 0.2430
Test Dice Crystal: 0.0000


  3%|▎         | 3/98 [00:18<09:27,  5.97s/it]

...

Test Dice: 0.5994
Test Dice Kidney: 0.5677
Test Dice Tumor: 0.4790


 99%|█████████▉| 97/98 [34:00<00:25, 25.63s/it]

Test Dice: 0.9281
Test Dice Kidney: 0.8938
Test Dice Tumor: 0.4682
Test Dice Crystal: 0.0000


100%|██████████| 98/98 [34:30<00:00, 21.13s/it]

Test Dice: 0.5244
Test Dice Kidney: 0.3817
Test Dice Tumor: 0.0182
Dice: 0.8004
Dice Kidney: 0.7989
Dice Tumor: 0.1360
Dice Cyst: 0.0000
